In [ ]:
#liberary
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from selenium import webdriver
import time
import datetime
import numpy as np
import ipywidgets as widgets
import glob
import os

In [ ]:
#Put 1 file in the folder, only read first one 
tracker_path='D:\\06.FullBeacon\\PO tracker'
file_list=glob.glob(tracker_path+'/*.xlsm')
PO_tracker=pd.read_excel(file_list[0],sheet_name='Report',skiprows=[0,2],header=[0],usecols='A,B,D,G,H,I,J,K,L,T,AO,AR,BC,BD,BE,BF,BG,BH,CC,CD,CE')
PO_tracker=PO_tracker.rename(columns = {'FB360 Commissioned':'Plan End Date','FB360 Commissioned.1':'Actual End Date'})

In [ ]:
Beacon_tracker=PO_tracker[(PO_tracker['Site Activity']=='Full Beacon') | (PO_tracker['Site Activity']=='2Step') | (PO_tracker['Site Activity']=='2Step Cold Build')]
Beacon_tracker

Creating the mark column for sorting

In [ ]:
#Classification of Beacon only or with 2step
def Classify(x):
    if '2Step'not in x:
        step_name=x+'2Step'
        if step_name in np.array(Beacon_tracker['DU ID']):
            return 'Full'
        else:
            return 'Beacon only'
    elif '2Step' in x:
        beacon_name=x.replace("2Step", "")
        if beacon_name in np.array(Beacon_tracker['DU ID']):
            return 'Attached'
        else:
            return '2Step only'
Beacon_tracker['Classification']=Beacon_tracker['DU ID'].apply(Classify)
Beacon_tracker.groupby('Classification').size()

In [ ]:
#Classification by Plan date
PO_date=2
cancel_point= pd.Timestamp.today() + pd.Timedelta(2, unit='M')
raise_point= pd.Timestamp.today() + pd.Timedelta(PO_date, unit='M')
today= pd.Timestamp.today()
def plan_status(row):
    if pd.notnull(row['Actual End Date']):
        return 'Finished'
    else:
        if pd.isnull(row['Plan End Date']):
            return 'de-instructed'
        elif row['Plan End Date'] > cancel_point:
            return 'Plan date postponed'
        elif ((row['Plan End Date'] <= raise_point)&(row['Plan End Date']>today)):
            return ('Process in '+str(PO_date)+' month')
        else:
            return 'Others'
Beacon_tracker['Plan_status'] = Beacon_tracker.apply (lambda row: plan_status(row),axis=1)    
Beacon_tracker.groupby('Plan_status').size()

In [ ]:
#Return related site to mysite column
def mysite(x):
    if '2Step'not in x:
        step_name=x+'2Step'
        if step_name in np.array(Beacon_tracker['DU ID']):
            return step_name
        else:
            return np.nan
    if '2Step' in x:
        beacon_name=x.replace("2Step", "")
        if beacon_name in np.array(Beacon_tracker['DU ID']):
            return beacon_name
        else:
            return np.nan
Beacon_tracker['Mysite']=Beacon_tracker['DU ID'].apply(mysite)

In [ ]:
#replace site ID to the site status
Partner=Beacon_tracker.copy()
Partner.set_index('DU ID',inplace=True)
Beacon_tracker['Mysite']=Beacon_tracker['Mysite'].map(Partner['Plan_status'])

In [ ]:
#Check PO status
Temp_active=Beacon_tracker['Active BTS I&C PO No.'].apply(lambda x: np.nan if 'Cancel' in str(x) else x)
Temp_passive=Beacon_tracker['Passive Partner Build PO'].apply(lambda x: np.nan if 'Cancel' in str(x) else x)
Need_2Step_PO=Beacon_tracker['Mysite'].apply(lambda x: True if x=='Finished'else False)
No_2Step_PO=pd.isnull(Beacon_tracker['Partner 2STEP to FB PO'])
Step_PO_noneed= ~(No_2Step_PO & Need_2Step_PO)
Beacon_tracker['PO_Covered']=((pd.notnull(Temp_active) | pd.notnull(Temp_passive)) & Step_PO_noneed)
Beacon_tracker.groupby(['Plan_status','Classification','PO_Covered']).size()

Creating Cancel Table(Full information) & PO_to_cancel(For cancel program)

In [ ]:
#Find Site needs to be cancelled, 1. not going 2. have PO 3.related site is not going ethier 
logic1=((Beacon_tracker['Plan_status']=='de-instructed') | (Beacon_tracker['Plan_status']=='Plan date postponed'))
logic2=Beacon_tracker['PO_Covered']
logic3=((Beacon_tracker['Mysite']=='de-instructed') | (Beacon_tracker['Mysite']=='Plan date postponed')| pd.isnull(Beacon_tracker['Mysite']))
Cancel_table=Beacon_tracker[logic1&logic2&logic3]
Cancel_table=Cancel_table.reset_index(drop=True)
Cancel_table

In [ ]:
#Final check, Attached should be same as Full
Cancel_table.groupby('Classification').size()

In [ ]:
#Remove attached site(PO is not on attached site)
logic4=(Cancel_table['Classification']!='Attached')
Temp_cancel=Cancel_table[logic4].reset_index(drop=True)
#Active PO
Active_list=pd.DataFrame(columns=['PO NO.','Site Code','Reason'])
Active_list['PO NO.']=Temp_cancel['Active BTS I&C PO No.']
Active_list['Site Code']=Temp_cancel['DU ID']
Active_list['Reason']=Temp_cancel['Plan_status']
#Passive PO
Passive_list=pd.DataFrame(columns=['PO NO.','Site Code','Reason'])
Passive_list['PO NO.']=Temp_cancel['Passive Partner Build PO']
Passive_list['Site Code']=Temp_cancel['DU ID']
Passive_list['Reason']=Temp_cancel['Plan_status']
#Combine Table and organize data
PO_to_cancel=Active_list.append(Passive_list)
PO_to_cancel=PO_to_cancel.dropna()
PO_to_cancel['PO NO.']=PO_to_cancel['PO NO.'].apply(lambda x : x.strip('(old)').strip())
PO_to_cancel['PO NO.']=PO_to_cancel['PO NO.'].apply(lambda x : x.strip('(new)').strip())
PO_to_cancel=PO_to_cancel.drop_duplicates()
PO_to_cancel=PO_to_cancel.sort_values('Reason').reset_index(drop=True)

Creating PO table

In [ ]:
logic5=(Beacon_tracker['Plan_status']==('Process in '+str(PO_date)+' month'))
PO_table=Beacon_tracker[logic5]

Writing in EXCEL

In [ ]:
Today=datetime.date.today()
Cancel_sheetname='Cancel'+Today.strftime('%d%m%Y')
PO_sheetname='PO'+Today.strftime('%d%m%Y')
Cancel_filename_with_route=r'D:\\06.FullBeacon\\FB_Manager_'+Today.strftime('%d%m%Y')+'.xlsx'
writer = pd.ExcelWriter(Cancel_filename_with_route)
PO_table.to_excel(writer,PO_sheetname)
Cancel_table.to_excel(writer,Cancel_sheetname)
PO_to_cancel.to_excel(writer,'sheet1')

writer.save()

PR_template filler under developing

In [ ]:
item_base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\FB_PR_item.xlsx'
AGF = pd.read_excel(item_base, sheet_name='AGF',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
ART = pd.read_excel(item_base, sheet_name='ART',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
AST = pd.read_excel(item_base, sheet_name='AST',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
RT = pd.read_excel(item_base, sheet_name='RT',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
ST = pd.read_excel(item_base, sheet_name='ST',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
GFM1 = pd.read_excel(item_base, sheet_name='GFM1',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
GFM2 = pd.read_excel(item_base, sheet_name='GFM2',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
GFM3 = pd.read_excel(item_base, sheet_name='GFM3',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
GFM4 = pd.read_excel(item_base, sheet_name='GFM4',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
GFM5 = pd.read_excel(item_base, sheet_name='GFM5',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
GFM6 = pd.read_excel(item_base, sheet_name='GFM6',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
MWR = pd.read_excel(item_base, sheet_name='MWR',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
RE2 = pd.read_excel(item_base, sheet_name='RE2',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
REWR = pd.read_excel(item_base, sheet_name='REWR',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')

In [ ]:
GFM6